In [1]:
import time
print(time.ctime(time.time()))

Mon Dec  9 20:53:08 2024


In [2]:
import os

import numpy as np
import pandas as pd

from collections import Counter
from pathlib import Path

In [3]:
# Day of calendar
day = 6

In [4]:
# Set path to input
input_dir = Path(os.path.abspath('')).parent

In [5]:
# Read input file
filepath = os.path.join(input_dir,'inputs','input_day%02d.txt' %(day))
#filepath = os.path.join(input_dir,'inputs','input_test.txt')
with open(filepath) as f:
    input = []
    for line in f:
        input.append(list(line.strip()))
grid = np.array(input)

In [6]:
grid

array([['.', '#', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '#', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '#', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U1')

## Part 1

Predict the path of the guard

In [7]:
# This will be our track matrix
guard_position = np.zeros_like(grid,dtype=int)

In [8]:
# First locate the guard's starting position
def start_idx(outer_list, target):
    for i, inner_list in enumerate(outer_list):
        if target in inner_list:
            j = inner_list.index(target)
            return i, j  # Return the row and column indices
    return None  # Return None if the element is not found

y1, x1 = np.where(grid=='^')
y1, x1 = y1[0], x1[0]

In [9]:
# Next step based on where the guard is looking
def next_step(face,x,y):
    if face == '^':
        dx, dy = (0,-1)
    elif face == '>':
        dx, dy = (1,0)
    elif face == 'v':
        dx, dy = (0,1)
    elif face == '<':
        dx, dy = (-1,0)

    xi = x + dx
    yi = y + dy

    return xi,yi

In [10]:
directions = ['^', '>', 'v', '<']
current_direction = '^'
x, y = x1, y1
guard_position[y,x] = 1

while not (x == 0 or x == grid.shape[1] - 1 or y == 0 or y == grid.shape[0] - 1):
    x_next, y_next = next_step(current_direction, x, y)

    # Check for obstacles
    while grid[y_next, x_next] == '#':
        # Update direction cyclically
        current_direction = directions[(directions.index(current_direction) + 1) % len(directions)]
        x_next, y_next = next_step(current_direction, x, y)

    # Record 
    # Move to the next valid position
    x, y = x_next, y_next
    guard_position[y,x] = 1


In [11]:
print(f'How many distinct positions will the guard visit before leaving the mapped area? -> {guard_position.sum()}')

How many distinct positions will the guard visit before leaving the mapped area? -> 5030


## Part 2

Lets add a single new obstruction and check if the guard is stuck in a loop

In [12]:
loop_matrix = np.full_like(guard_position, fill_value=0)

In [13]:
loop_count = 0
# Loop through every single location
for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
        # Place new obstacle if it is possible
        if not grid[i,j] == '.':
            continue

        new_grid = grid.copy()
        new_grid[i,j] = '#' # place new obstacle
        loop = False

        # Reset guard position and direction
        current_direction = '^'
        x, y = x1, y1
        # This will be our track matrix
        guard_position = np.zeros_like(grid,dtype=int)
        guard_position[y,x] = 1
        guard_direction = np.empty_like(guard_position, dtype=str)
        guard_direction[y,x] = current_direction

        while not (x == 0 or x == grid.shape[1] - 1 or y == 0 or y == grid.shape[0] - 1) and not loop:
            x_next, y_next = next_step(current_direction, x, y)

            # Check for obstacles
            while new_grid[y_next, x_next] == '#':
                # Update direction cyclically
                current_direction = directions[(directions.index(current_direction) + 1) % len(directions)]
                x_next, y_next = next_step(current_direction, x, y)

            # Check if the guard has already visited the next position facing the same direction
            if guard_position[y_next,x_next] == 1 and guard_direction[y_next,x_next] == current_direction:
                loop_count += 1
                loop_matrix[i,j] = 1
                loop = True


            # Record 
            # Move to the next valid position
            x, y = x_next, y_next
            guard_position[y,x] = 1
            guard_direction[y,x] = current_direction

In [14]:
print(f'How many different positions could you choose for this obstruction? -> {loop_count}')

How many different positions could you choose for this obstruction? -> 1928
